In [13]:
!pip install nltk
!pip install datasets
!pip install transformers
!pip install numpy
!pip install wandb
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import wandb
wandb.init(project="title_prediction", entity="a988")

eval/loss,█▇▆▆▆▆▁
eval/rouge1,▁▂▃▅▄▄█
eval/rouge2,▁▂▃▄▃▃█
eval/rougeL,▁▂▃▄▄▄█
eval/rougeLsum,▁▂▃▄▄▄█
eval/runtime,▇▁█▅▆█▄
eval/samples_per_second,▂█▁▄▃▁▅
eval/steps_per_second,▂█▁▄▃▁▅
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███▁
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁


In [9]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from datasets import load_dataset,load_from_disk,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from google.colab import drive
drive.mount('/content/drive')

def data_preparation():
    full_dataset = load_dataset("csv", data_files='/content/drive/MyDrive/title_generation_final/train.csv')
    cols_to_remove = list(full_dataset['train'].features.keys())
    cols_to_remove.remove("title")
    cols_to_remove.remove("text")
    data=full_dataset.remove_columns(cols_to_remove)
    dataset = data['train'].train_test_split(test_size=0.1)
    test_val = dataset['test'].train_test_split(test_size=0.5)
    dataset['val'] = test_val['train']
    dataset['test'] = test_val['test']
    dataset.save_to_disk('/content/drive/MyDrive/title_generation_final/title_generation_dataset')
    dataset = load_from_disk('/content/drive/MyDrive/title_generation_final/title_generation_dataset')
    nltk.download('punkt')
    return dataset

def tokenizer_():
    tokenizer = AutoTokenizer.from_pretrained('t5-base')
    return tokenizer

def preprocess_data(data,MAX_SOURCE_LEN = 512,MAX_TARGET_LEN = 128):    
    # Initialize T5-base tokenizer
    tokenizer=tokenizer_()
    model_inputs = tokenizer(data['text'], max_length=MAX_SOURCE_LEN, padding=True, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(data['title'], max_length=MAX_TARGET_LEN, padding=True, truncation=True)
    # Replace all pad token ids in the labels by -100 to ignore padding in the loss
    labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
    model_inputs['labels'] = labels["input_ids"]
    return model_inputs

def run_process_to_data_preprocess():
    data=data_preparation()
    processed_dataset = data.map(
        preprocess_data,
        batched=True,
        remove_columns=['text', 'title'],
        desc="Running tokenizer on dataset",)
    return processed_dataset

batch_size = 4
num_epochs = 2
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 1000
lr_scheduler_type = "linear"

# Define training arguments
def training_params():
  training_args = Seq2SeqTrainingArguments(
      output_dir="/content/drive/MyDrive/title_generation_final/model-t5-base",
      evaluation_strategy="steps",
      eval_steps=eval_every,
      learning_rate=learning_rate,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      weight_decay=weight_decay,
      save_steps=6500,
      save_total_limit=3,
      num_train_epochs=num_epochs,
      predict_with_generate=True,
      logging_steps=log_every,
      group_by_length=True,
      lr_scheduler_type=lr_scheduler_type,
      report_to="wandb",
      resume_from_checkpoint=True,
  )
  return training_args

def model_():
    model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
    return model

def train_initialize(eval_pred):
  # Initialize T5-base model
    metric = load_metric("rouge")
    predictions, labels = eval_pred
    tokenizer=tokenizer_()
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores and get the median scores
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

# Dynamic padding in batch using a data collator
def train():
  print("PROCESSED DATASET RUNNING")
  processed_dataset=run_process_to_data_preprocess()
  print("TOKENIZER RUNNING")
  tokenizer=tokenizer_()
  print("MODEL RUNNING")
  model=model_()
  print("TRAINNG ARGS RUNNING")
  training_args=training_params()
  data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
  # Define the trainer
  trainer = Seq2SeqTrainer(
      model,
      training_args,
      train_dataset=processed_dataset["train"],
      eval_dataset=processed_dataset["val"],
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=train_initialize,
  )
  trainer.train()
  model_dir="/content/drive/MyDrive/title_generation_final/models/"
  trainer.save_model(model_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
if __name__=="__main__":
  data_preparation()
  train()

  0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


PROCESSED DATASET RUNNING


  0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,

...

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1000,1.865400,1.625453,47.743200,21.886700,42.779100,42.896100


***** Running Evaluation *****
  Num examples = 750
  Batch size = 4
<ipython-input-9-391e4e8a4283>:87: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1000,1.865400,1.625453,47.743200,21.886700,42.779100,42.896100
2000,1.800300,1.579411,48.224100,22.549900,43.652600,43.750500
3000,1.685800,1.539798,49.336700,23.517700,44.508000,44.579800
4000,1.551100,1.541414,50.628800,24.578500,45.748000,45.842900
5000,1.549500,1.535670,50.319200,24.123200,45.516000,45.610700
6000,1.430500,1.519827,50.344500,23.981800,45.395500,45.442100


***** Running Evaluation *****
  Num examples = 750
  Batch size = 4
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summariz

In [11]:
model_trained = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/').to('cuda')
token_trained = AutoTokenizer.from_pretrained('/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/')


def test():
  print("PROCESSED DATASET RUNNING")
  processed_dataset=run_process_to_data_preprocess()
  print("TOKENIZER RUNNING")
  tokenizer=token_trained
  print("MODEL RUNNING")
  model=model_trained
  print("TRAINNG ARGS RUNNING")
  testing_args=training_params()
  data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
  # Define the trainer
  tester = Seq2SeqTrainer(
      model,
      testing_args,
      train_dataset=processed_dataset["train"],
      eval_dataset=processed_dataset["val"],
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=train_initialize,
  )
  return tester

tester=test()
processed_dataset=run_process_to_data_preprocess()
tester.evaluate(eval_dataset=processed_dataset['test'])

loading configuration file /content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_re

PROCESSED DATASET RUNNING


  0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Running tokenizer on dataset:   0%|          | 0/14 [00:00<?, ?ba/s]

...

TOKENIZER RUNNING
MODEL RUNNING
TRAINNG ARGS RUNNING


  0%|          | 0/1 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
***** Running Evaluation *****
  Num examples = 750
  Batch size = 4
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,


{'eval_loss': 1.2756199836730957,
 'eval_rouge1': 53.1058,
 'eval_rouge2': 29.134,
 'eval_rougeL': 49.15,
 'eval_rougeLsum': 49.1714,
 'eval_runtime': 114.874,
 'eval_samples_per_second': 6.529,
 'eval_steps_per_second': 1.637}

In [20]:
temperature = 0.9
num_beams = 4
max_gen_length = 128
from rouge_score import rouge_scorer

In [21]:
text='''(Reuters) - U.S. President Donald Trump faces an temporary hold on his travel ban on seven mainly Muslim countries, but the outcome of a ruling on the executive order’s ultimate legality is less certain.  Any appeals of decisions by U.S. District Court Judge James Robart in Seattle face a regional court dominated by liberal-leaning judges who might not be sympathetic to Trump’s rationale for the ban, and a currently shorthanded Supreme Court split 4-4 between liberals and conservatives. The temporary restraining order Robart issued on Friday in Seattle, which applies nationwide, gives him time to consider the case in more detail, but also sends a signal that he is likely to impose a more permanent injunction.    The Trump administration has appealed that order. The San Francisco-based 9th U.S. Circuit Court of Appeals said late on Saturday that it would not decide whether to lift the judge’s ruling, as requested by the U.S. government, until it receives briefs from both sides, with the administration’s filing due on Monday. Appeals courts are generally leery of upending the status quo, which in this case - for now - is the suspension of the ban.  The upheaval prompted by the new Republican administration’s initial announcement of the ban on Jan. 27, with travelers detained at airports upon entering the country, would potentially be kickstarted again if Robart’s stay was lifted. The appeals court might also take into account the fact that there are several other cases around the country challenging the ban. If it were to overturn the district court’s decision, another judge somewhere else in the United States could impose a new order, setting off a new cascade of court filings.  If the appeals court upholds the order, the administration could immediately ask the U.S. Supreme Court to intervene. But the high court is generally reluctant to get involved in cases at a preliminary stage, legal experts said. The high court is short one justice, as it has been for a year, leaving it split between liberals and conservatives. Any emergency request by the administration would need five votes to be granted, meaning at least one of the liberals would have to vote in favor.  “I think the court’s going to feel every reason to stay on the sidelines as long as possible,” said Steve Vladeck, a professor at the University of Texas School of Law. Trump last week nominated a conservative appeals court judge, Neil Gorsuch, to fill the vacancy, but he will not be sitting on the Supreme Court for at least two months. Gorsuch’s vote, if he is confirmed by the U.S. Senate, could come into play if the case were to reach the court at a later stage of the litigation. Once the case proceeds past the injunction stage of the litigation and onto the merits of whether the order is legally sound, legal experts differ over how strong the government’s case would be.  Richard Primus, a professor of constitutional law at the University of Michigan Law School, said the administration could struggle to convince courts that the ban was justified by national security concerns.  The Supreme Court has previously rejected the idea that the government does not need to offer a basis for its actions in the national security context, including the landmark 1971 Pentagon Papers case, in which the administration of President Richard Nixon tried unsuccessfully to prevent the press from publishing information about United States policy toward Vietnam. “The government’s argument so far in support of the order is pretty weak,” Primus said. Jonathan Adler, a professor at Case Western Reserve University School of Law, said the administration has legal precedent on its side, with the courts generally deferential to executive action on immigration.  However, he said it is unusual for the courts to be asked to endorse “a policy that appears to have been adopted in as kind of haphazard and arbitrary way as this one appears to have been.” '''
inputs = token_trained([text], max_length=512, return_tensors='pt',truncation=True)

title_ids = model_trained.generate(
    inputs['input_ids'].to('cuda'), 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True
)
title = token_trained.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(title)

Trump travel ban appeals face regional court, liberal-leaning Supreme Court


In [22]:
def evaluate(model_output,actual_titles):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = list()
    for output,actual in zip(model_output,actual_titles):
        s = scorer.score(output,actual)
        scores.append(s['rouge1'].fmeasure)
    print('Evaluation result',np.mean(scores))
    return scores

In [23]:
evaluate("Trump faces uphill battle to overturn judge's travel ban stay","Trump faces uphill battle to overcome court's hold on travel ban")

Evaluation result 0.47540983606557374


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [24]:
data=pd.read_csv("/content/drive/MyDrive/title_generation_final/train.csv")
data=data[["text","title"]]
data=data.iloc[:2,:]

In [27]:
import pandas as pd
from rouge_score import rouge_scorer
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_trained = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/').to('cuda')
token_trained = AutoTokenizer.from_pretrained('/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/')
submission = pd.read_csv('/content/drive/MyDrive/title_generation_final/train.csv')
data=submission.loc[:20,:]
print(data)
def predict(texts,temperature=0.6,num_beams = 4,max_gen_length = 256):
    # write code to output a list of title for each text input to the predict method
    data=texts.tolist()
    inputs = token_trained(data, max_length=512, return_tensors='pt',truncation=True, padding=True)
    title_ids = model_trained.generate(
    inputs['input_ids'].to('cuda'), 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True)
    print(title_ids)
    # a=[title_ids[i] for i in range(len(title_ids))]
    # print(len(a))
    pred_title=[]
    for i in range(len(data)):
      title = token_trained.decode(title_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)
      pred_title.append(title)
    return pred_title

def test_model():
    pred = predict(data["text"])
    data['predicted_title'] = pred
    data.to_csv('submission.csv',index=False)


def evaluate(model_output,actual_titles):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = list()
    for output,actual in zip(model_output,actual_titles):
        s = scorer.score(output,actual)
        scores.append(s['rouge1'].fmeasure)
    print('Evaluation result',np.mean(scores))
    return scores

loading configuration file /content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/title_generation_final/model-t5-base/checkpoint-6500/",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_re

In [1]:
if __name__=="__main__":
    #write model loading code here
    test_model()